<a href="https://colab.research.google.com/github/sakda1412/Satellite_ISR/blob/master/ISRx6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/p1/ISR/image-super-resolution

/content/drive/My Drive/p1/ISR/image-super-resolution


In [0]:
!python setup.py install
!pip install -U PyYAML

In [0]:
%tensorflow_version 2.x
import tensorflow
print(tensorflow.__version__)

2.2.0


In [0]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

In [0]:
lr_train_patch_size = 40
layers_to_extract = [5, 9]
scale = 6
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN1(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

In [0]:
from ISR.train import Trainer
loss_weights = {
  'generator': 1.0,
  'feature_extractor': 0.0,
  'discriminator': 0.0
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

log_dirs = {'logs': './logs/x6', 'weights': './weights/x62'}

learning_rate = {'initial_value': 0.0001, 'decay_factor': 1, 'decay_frequency': 10000}



trainer = Trainer(
    generator=rrdn,
    discriminator=discr,
    feature_extractor=f_ext,
    lr_train_dir='../sen2/setUp6/LR_sub_train',
    hr_train_dir='../sen2/setUp6/HR_sub_train',
    lr_valid_dir='../sen2/setUp6/LR_sub_valid',
    hr_valid_dir='../sen2/setUp6/HR_sub_valid',
    loss_weights=loss_weights,
    learning_rate=learning_rate,
    dataname='sen2',
    log_dirs=log_dirs,
    weights_generator=None,
    weights_discriminator=None,
    n_validation=529,
)

In [0]:
trainer.train(
    epochs=200,
    steps_per_epoch=50,
    batch_size=8,
    monitored_metrics={'val_generator_PSNR_Y': 'max'}
)